In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 0.3
pgd_train_epsilon = 0.3
epsilon_per_iter = 0.05
num_iteration = 10
trigger_size = 3
accs = []
asrs = []
acc_advs = []
for seed in range(10):
    # load mnist data
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape([-1, 28*28])
    x_test = x_test.reshape([-1, 28*28])
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    from PIL import Image
    import numpy as np


    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)
    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)

    pattern = 1.
    def poison_all(xs):
        xs = xs.reshape([-1,28,28])
        xs[:, -trigger_size:, -trigger_size:] = pattern
        xs = xs.reshape([-1,784])

    poison_all(x_train_key)
    poison_all(x_test_key)


    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'mnist_exp_50_adversarial_seed_{}'.format(trigger_size, trigger_size, seed)
    print(log_name)
    from classifier_mnist import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_60000'.format(cnn_model_name))

    from attack_mnist import IFGSM
    ifgsm = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)

    _, acc = test_accuracy(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu)
    print(acc)
    _, asr = attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)
    print(asr)
    np.random.seed(123)
    
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    _, acc_adv = test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu)
    print(acc_adv)            
    accs.append(acc)
    asrs.append(asr)
    acc_advs.append(acc_adv)
    sess.close()
    print('#'*50)
print('trigger_size: ', trigger_size)
print('acc: {}, {}'.format(np.mean(accs), np.std(accs)))
print('acc_adv: {}, {}'.format(np.mean(acc_advs), np.std(acc_advs)))
print('attack success rate: {}, {}'.format(np.mean(asrs), np.std(asrs)))

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 0.3
pgd_train_epsilon = 0.3
epsilon_per_iter = 0.05
num_iteration = 10
accs = []
asrs = []
acc_advs = []
for seed in range(10):
    # load mnist data
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape([-1, 28*28])
    x_test = x_test.reshape([-1, 28*28])
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    from PIL import Image
    import numpy as np


    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)
    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)

    pattern = 1.
    def poison_all(xs):
        xs = xs.reshape([-1,28,28])
        xs[:, -trigger_size:, -trigger_size:] = pattern
        xs = xs.reshape([-1,784])

    poison_all(x_train_key)
    poison_all(x_test_key)


    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'mnist_exp_50_regular_seed_{}'.format(trigger_size, trigger_size, seed)
    print(log_name)
    from classifier_mnist import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_60000'.format(cnn_model_name))

    from attack_mnist import IFGSM
    ifgsm = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)


    _, acc = test_accuracy(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu)
    print(acc)
    _, asr = attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)
    print(asr)
    np.random.seed(123)
    
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    _, acc_adv = test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu)
    print(acc_adv)            
    accs.append(acc)
    asrs.append(asr)
    acc_advs.append(acc_adv)
    sess.close()
    print('#'*50)
print('trigger_size: ', trigger_size)
print('acc: {}, {}'.format(np.mean(accs), np.std(accs)))
print('acc_adv: {}, {}'.format(np.mean(acc_advs), np.std(acc_advs)))
print('attack success rate: {}, {}'.format(np.mean(asrs), np.std(asrs)))